In [ ]:
"""
How do you generate sequence data?

1. The universal way to generate sequence data in deep learning is to train a network (usually an RNN or a convnet) to predict the next
   token or next few tokens in a sequence, using the previous tokens as input..As usual when working with text data, tokens are typically
   words or characters, and any network that can model the probability of the next token given the previous ones is called a language model.
   A language model captures the latent space of language: its statistical structure.

2.  Once you have such a trained language model, you can sample from it (generate new sequences): you feed it an initial string of text
    (called conditioning data), ask it to generate the next character or the next word (can even generate several tokens at once), add
    the generated output back to the input data, and repeat the process many times. This loop allows you to generate sequences of arbitrary
    length that reflect the structure of the data on which the model was trained: sequences that look almost like human-written sentences.
"""

In [5]:
"""
Sampling strategy


1. When generating text, the way you choose the next character is crucially important. A naive approach is greedy sampling, consisting of
   always choosing the most likely next character. But such an approach results in repetitive, predictable strings that don’t look like
   coherent language. A more interesting approach makes slightly more surprising choices: it introduces randomness in the sampling process,
   by sampling from the probability distribution for the next character. This is called stochastic sampling. In such a setup, if e has
   a probability 0.3 of being the next character, according to the model, you’ll choose it 30% of the time.

2. Sampling probabilistically from the softmax output of the model is neat: it allows even unlikely characters to be sampled some of
   the time, generating more interesting looking sentences and sometimes showing creativity by coming up with new, realistic-sounding
   words that didn’t occur in the training data. But there’s one issue with this strategy: it doesn’t offer a way to control
   the amount of randomness in the sampling process.

3. Why would you want more or less randomness? Consider an extreme case: pure random sampling, where you draw the next character from
   a uniform probability distribution, and every character is equally likely. This scheme has maximum randomness; in other words, this
   probability distribution has maximum entropy. Naturally, it won’t produce anything interesting. At the other extreme, greedy sampling
   doesn’t produce anything interesting, either, and has no randomness: the corresponding probability distribution has minimum entropy.
   Sampling from the “real” probability distribution — the distribution that is output by the model’s softmax function — constitutes
   an intermediate point between these two extremes. But there are many other intermediate points of higher or lower entropy that you may
   want to explore. Less entropy will give the generated sequences a more predictable structure (and thus they will potentially be more
   realistic looking), whereas more entropy will result in more surprising and creative sequences. When sampling from generative models,
   it’s always good to explore different amounts of randomness in the generation process. Because we—humans—are the ultimate judges
   of how interesting the generated data is, interestingness is highly subjective, and there’s no telling in advance where the point of
   optimal entropy lies.

4.  In order to control the amount of stochasticity in the sampling process, here introduces a parameter called the softmax temperature
    that characterizes the entropy of the probability distribution used for sampling: it characterizes how surprising or predictable
    the choice of the next character will be. Given a temperature value, a new probability distribution is computed from the original
    one (the softmax output of the model) by reweighting it.

5. Higher temperatures result in sampling distributions of higher entropy that will generate more surprising and unstructured generated
   data, whereas a lower temperature will result in less randomness and much more predictable generated data.
"""
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    # original_distribution is a 1D Numpy array of probability values that must sum to 1.
    # temperature is a factor quantifying the entropy of the output distribution.
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # Returns a reweighted version of the original distribution. The sum of the distribution may no longer
    # be 1, so you divide it by its sum to obtain the new distribution.
    return distribution / np.sum(distribution)

In [8]:
"""
Implementing Character-level LSTM Text Generation (1)

"""
import tensorflow.keras

# download the data
path = tensorflow.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

# vectorizing sequences of characters
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])      # extracted sentence
    next_chars.append(text[i + maxlen])        # target
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Corpus length: 600893
Number of sequences: 200278
Unique characters: 57
Vectorization...


In [11]:
"""
Implementing Character-level LSTM Text Generation (2)

(*) Generate new text by doing the following repeatedly
    -- Draw from the model a probability distribution for the next character, given the generated text available so far.
    -- Reweight the distribution to a certain temperature.
    -- Sample the next character at random according to the reweighted distribution.
    -- Add the new character at the end of the available text.
"""
from tensorflow.keras import layers

# build the network
model = tensorflow.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

# compile the model
optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model.summary())

# sampling function: sample the next character given the model’s predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
"""
Implementing Character-level LSTM Text Generation (2)

1. Generate new text by doing the following repeatedly
   -- Draw from the model a probability distribution for the next character, given the generated text available so far.
   -- Reweight the distribution to a certain temperature.
   -- Sample the next character at random according to the reweighted distribution.
   -- Add the new character at the end of the available text.

2. A low temperature value results in extremely repetitive and predictable text, but local structure is highly realistic: in particular,
   all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text becomes
   more interesting, surprising, even creative; it sometimes invents completely new words that sound somewhat plausible (such as eterned
   and troveration). With a high temperature, the local structure starts to break down, and most words look like semi-random strings of
   characters. Without a doubt, 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment with
   multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

3. Note that by training a bigger model, longer, on more data, you can achieve generated samples that look much more coherent and realistic
   than this one. But, of course, don’t expect to ever generate any meaningful text, other than by random chance: all you’re doing is
   sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there’s
   a distinction between what communications are about and the statistical structure of the messages in which communications are encoded. 
"""
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]  # random seed text
    print('--- Generating with seed: "' + generated_text + '"')
    
    # try a range of different sampling temperatures
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
200278/200278 [==============================] - 90s 448us/step - loss: 1.9677
--- Generating with seed: "heir mission, their hard, involuntary, imperative mission (i"
------ temperature: 0.2
heir mission, their hard, involuntary, imperative mission (in the present of the been and the contine the proble the spirits of the strenge and the for the present of the present and the present and be conterality of the man and the faitions of the spirits of the conterning to the still the strenge in the strenge the precession of the spirits of the still the spirits of the was the strenge in the spirits of the spirits of the spirits of the spirit and stre------ temperature: 0.5
spirits of the spirits of the spirits of the spirit and strenger it is a more even a mare. hitherter and the decture of the voreant and call the relision of the the soulness in the the were an it as the beto the conternition of the higher man for the are one the all the order and onere or a sublitions of 

mortaful, and to holighing deecdaby imientengers arise with things smoungs-heearm; becomate hifd, whenever castan
axoth) parittiones fautomeres not be, who apeer cper cprese higher had us goognis facanthing
groud arigilan
seleed--2. anian-salmathings, more gets bestment diving
computions that
theirgnarob.
man who, more
femaus ensliturnity
differating,w"g againl himledepoch 5
Epoch 1/1
200278/200278 [==============================] - 88s 439us/step - loss: 1.4596
--- Generating with seed: "l metaphysic. without
the dream, men would never have been i"
------ temperature: 0.2
l metaphysic. without
the dream, men would never have been in the sense of the consideration of the self-case of the same all the same as the self-constand of the sense of the world to the self-conceived to be soul as the structual and same and such a soul as the reason of the soul as the sease of the same the deception of the self-conception of the self-contentay and same the wholly of the same and present the moral

for the beautifor, and in some
things:
-again., liter-from upon thek thallome
equally inplay as the rable and
facice regard inorics of mock, but me which flow
o) as
well delighes da------ temperature: 1.2
rd inorics of mock, but me which flow
o) as
well delighes dangeity, with there, "dogpidelyy men is nighth
you immeast as l uneocile: this one and
world., a fiasury: this does thehsely of enlifick of many, to incentioss," undenauity--would too sasssimess
look ran as
being, in
wice thecy tolies, understous.--"a-vance of pall shricgot
of the refadiness of estifieia bletion, yoo.

2thouch tedea, sox therepreded brutter himself--assumed a cowne-weathraphess; leepoch 9
Epoch 1/1
200278/200278 [==============================] - 88s 439us/step - loss: 1.3944
--- Generating with seed: "higher entity than the latter? is the former not palpably a
"
------ temperature: 0.2
higher entity than the latter? is the former not palpably a
moral sure and the self-to the comprehensionage and the live that

the german from a superiority and and all those the sciences more seems is the daisstant instincted to disgualone in every
tyophinimes. matlus, general and kildunity of itspends period and that the action--i artage: i have nothing?" of a god; if still nonests us benevolent, every defilation for its least, he has also kil human cling old philosopher occasion them itself the furmovation of higher various"grean go wookh together,
educations, a
pobte in creati------ temperature: 1.2
rious"grean go wookh together,
educations, a
pobte in creatiat it will be differ, morkaitign and ag"
is fakal
estuatines, it all can sympathice, a perstances mostely some kindarowly of plays spitiouely, too risift.
     te, him cloue absedent, and not awayd
mest swaringly tart, is a suddations of these, we mind it
is
tere regard to that (there is a have rich mins
positions, he
simstrong. itself, in maninetiesore as actoution is agaies under, a malicosed haepoch 13
Epoch 1/1
200278/200278 [======================

/data/home/zhangmu/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log


...

uch a means of the conscience of the senses of the sense of the power, and also the democratic of the suparistich is much and the works of the precisely what is learned a really on a station and the self-destruction of power and not be the same such a comparatoures of the existence of the faith of the cause of the sense of the religious tempo of will, as a more new and himself what is society of the sake of means as the suffering of strength in the present------ temperature: 1.0
he sake of means as the suffering of strength in the present fatherlan, as a personal mens
from one another a beiling
-he, who can be a cadene in some supposible and in the fine true complexce of the gooting without mimached as everykself--shothes pultt the moves whaterrearly how to to being with bad formult; he say up
there ponism only in what our ornens.



19

=this fissong
of cihcurily increases alsom in loved to oy weakest for the
same serviled; we do------ temperature: 1.2
es alsom in loved to oy weakest 

for people like himself, and for that which they require--for the same any states of the senses of the discordines of the self-contrary the states of the superiority of the superiority of the scientific priderishment of the subjective of the state of the senses of the scientific man who are the sense of the same antiquity of the states of the sure an opposite of the strength of the same antiquity of the superiority of the superiority of the sense of the sa------ temperature: 0.5
of the superiority of the superiority of the sense of the same will to be the successfully and causate in aspity and light consequence in the sense, and for their means of the harmer sympathy has not assume and not all recoles a man is the sense
of such a higher to say, in a sure antugemined from the and man has a man as if the ratired for the sach in the same souls, the last faciolity of the seriousness this prider of the feeling of the world has been eno------ temperature: 1.0
ousness this prider of the feeli

for tamber
of uping, who
has
opinion--and
one owing to despair, a kind apparentlyness in reepoch 49
Epoch 1/1
200278/200278 [==============================] - 88s 439us/step - loss: 1.2866
--- Generating with seed: " disposed to
concede such a thing--has to do not with its na"
------ temperature: 0.2
 disposed to
concede such a thing--has to do not with its nature the same and and strongly and science and science and by the same the strongly, which is a standard with the struggle and destination of the struggle and the consideration of the words of the deals the present schopenhauer's in the same the spirit of the same and the strongest and and all the conscience of the same any best more the conscience of the consideration of the present strongest str------ temperature: 0.5
conscience of the consideration of the present strongest stronger and promises of the spirit in the categor of the exception of his superstition and concerns and self-effect in the religious or and with the same at

most gold "l"''hirlily estimate attains
grfusiological matter has had ever alone.

227. the reat, what materins, as have that we
things, these questand
about when. we read, where a unitabed visivuure-"in.
u=just freme tarkagtly.

1. murked still
undeed have to do anything and traks sympathy realnd our fated bring in each clyset
pure of
"
standalion by years laughtenesists place of shfarestressstening in
plurepoch 53
Epoch 1/1
200278/200278 [==============================] - 88s 439us/step - loss: 1.2842
--- Generating with seed: "mand respect as a triumph of reason: and did
in fact command"
------ temperature: 0.2
mand respect as a triumph of reason: and did
in fact commanding and consideration of the same the stronges the present self-contrary and consideration and states of the same the free spirit in the same time the same time a sense of the world and the same the former and are also the interest of the same the present dispertaness and the moral present and present the same the se

from rest:on to instorm in be antiquity of many, as it were a all "farfting an estantopative," us, we consists with the garme. always tests which hanbern, they redulse to different
"weaker
conduct in many do too a way of exact tests--the soul straigidetion: "if he emotioning in himself?
overl ------ temperature: 1.2
the soul straigidetion: "if he emotioning in himself?
overl people to get to the idea,
with it focher, such a monish,
his grour had because it makes
semmative decluses every
wanter, morry cavel richer, is worter-whin menite etsing in the ekesn.


1ad=ack, generately: the exhenarm
hab, not in appread the cruelty, determined france. why diccless
b. more able masden a cnogatpate
formeths of
fakvations, will perceive to whok-as peropopite in iveras
questions eepoch 57
Epoch 1/1
200278/200278 [==============================] - 88s 438us/step - loss: 1.2836
--- Generating with seed: "xhaustion of their will to
live (their nerves). they employ "
------ temperature: 0.2
xhaustion o